Many factors that affect how much you pay for health insurance are not within your control. Nonetheless, it's good to have an understanding of what they are. Here are some factors that affect how much health insurance premiums cost

* age: age of primary beneficiary

* sex: insurance contractor gender, female, male

* bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

* children: Number of children covered by health insurance / Number of dependents

* smoker: Smoking

* region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest

predict `charges`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from sklearn.linear_model import Ridge

url = "/work/data/homework 25/insurance.csv"
df = pd.read_csv(url)
df.info()
df["region"].unique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [ ]:
2505

In [ ]:
df["age"].desr

18    69
19    68
20    29
51    29
45    29
46    29
47    29
48    29
50    29
52    29
28    28
54    28
21    28
27    28
26    28
49    28
25    28
24    28
23    28
22    28
53    28
42    27
44    27
43    27
41    27
40    27
31    27
30    27
29    27
56    26
34    26
33    26
32    26
57    26
55    26
35    25
59    25
58    25
36    25
39    25
38    25
37    25
60    23
61    23
62    23
63    23
64    22
Name: age, dtype: int64

In [ ]:
df["charges"].describe()

count     1338.000000
mean     13270.422265
std      12110.011237
min       1121.873900
25%       4740.287150
50%       9382.033000
75%      16639.912515
max      63770.428010
Name: charges, dtype: float64

In [ ]:
!pip install -q xlrd
!pip install -q optuna
!pip install -q openpyxl

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, StandardScaler, PowerTransformer,Normalizer
import pandas as pd
import datetime as dt
import optuna

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
def load_data():
    df=pd.read_csv(url)
    X=df.drop(["charges"],axis = 1)
    y=df["charges"]
    le = LabelEncoder()
    X.iloc[:, 1] = le.fit_transform(X.iloc[:, 1])
    X.iloc[:, 4] = le.fit_transform(X.iloc[:, 4])
    ct = ColumnTransformer(transformers=[('OneHotEncoder', OneHotEncoder(), [5])], 
                            transformers=[('label', LabelEncoder(), [2])], 
                        remainder='passthrough')
    X = np.array(ct.fit_transform(X))   
    return X,y

X,y = load_data()    


In [ ]:

le = LabelEncoder()
X.iloc[:, 1] = le.fit_transform(X.iloc[:, 1])
X.iloc[:, 4] = le.fit_transform(X.iloc[:, 4])
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [5])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=30,random_state=True,shuffle=True)

scaler =MinMaxScaler()
scaler.fit(X_train)
model=LinearRegression()
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)
model.fit(scaled_X_train,y_train)
prediction = model.predict(scaled_X_test)
print('MAE:', metrics.mean_absolute_error(y_test, prediction))



MAE: 3451.7934073333336


In [ ]:
def load_data():
    
    df=pd.read_csv(url)
    X=df.drop(["charges"],axis = 1)
    y=df["charges"]
    le = LabelEncoder()
    X.iloc[:, 1] = le.fit_transform(X.iloc[:, 1])
    X.iloc[:, 4] = le.fit_transform(X.iloc[:, 4])
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [5])], remainder='passthrough')
    X = np.array(ct.fit_transform(X))   
    
    return X,y


def objective(trial):

    
    regressor_name = trial.suggest_categorical("regressors", ["SVR", "RandomForestRegressor","DecisionTreeRegressor","LinearRegression"])
    if regressor_name == "SVR":
        svr_c = trial.suggest_float("svr_c", 1e-3, 1e3, log=True)
        model = SVR(C=svr_c)
    elif regressor_name == "RandomForestRegressor":
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 12, log=True)
        model = RandomForestRegressor(max_depth=rf_max_depth, 
                                                       n_estimators=10)
    elif regressor_name == "DecisionTreeRegressor":
        dt_criteria = trial.suggest_categorical('criterion', ['mse', 'friedman_mse'])
        dt_max_depth = trial.suggest_int("dt_max_depth", 2, 12)
        model = DecisionTreeRegressor(criterion= dt_criteria,
                                                   max_depth=dt_max_depth)
    elif regressor_name == 'LinearRegression':
        model = LinearRegression() 
    

    scaler_string = trial.suggest_categorical("------------------------------------_scaler",["no_scaler", "StandardScaler","RobustScaler","MinMaxScaler", "MaxAbsScaler", "StandardScaler", "PowerTransformer","Normalizer"])    
    if scaler_string == "no_scaler":
        scaled_X = X
    else:
        scaler = eval(scaler_string)()
        scaler.fit(X)
        scaled_X = scaler.transform(X)

    cv=KFold(shuffle=True,random_state=42)
    score=cross_val_score(model,scaled_X,y,cv=cv,scoring="neg_mean_absolute_error")
    trial_score = score.mean() 

    return trial_score

X,y = load_data()
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)
print(study.best_trial)

[I 2021-10-03 19:49:59,995] A new study created in memory with name: no-name-9e99a18e-0c9d-49fa-ba98-99e7f5a7bed8
[I 2021-10-03 19:50:00,129] Trial 0 finished with value: -2738.0076330499883 and parameters: {'regressors': 'RandomForestRegressor', 'rf_max_depth': 3, '------------------------------------_scaler': 'StandardScaler'}. Best is trial 0 with value: -2738.0076330499883.
[I 2021-10-03 19:50:00,689] Trial 1 finished with value: -4590.214800963794 and parameters: {'regressors': 'SVR', 'svr_c': 184.26378042142096, '------------------------------------_scaler': 'MinMaxScaler'}. Best is trial 0 with value: -2738.0076330499883.
[I 2021-10-03 19:50:01,233] Trial 2 finished with value: -5832.876694592005 and parameters: {'regressors': 'SVR', 'svr_c': 116.95708085109072, '------------------------------------_scaler': 'MinMaxScaler'}. Best is trial 0 with value: -2738.0076330499883.
[I 2021-10-03 19:50:01,252] Trial 3 finished with value: -2756.713735740176 and parameters: {'regressors': 

In [ ]:

def objective(trial):

    
    regressor_name = trial.suggest_categorical("regressors", ["SVR", "RandomForestRegressor","DecisionTreeRegressor","LinearRegression"])
    if regressor_name == "SVR":
        svr_c = trial.suggest_float("svr_c", 1e-3, 1e3, log=True)
        model = SVR(C=svr_c)
    elif regressor_name == "RandomForestRegressor":
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 12, log=True)
        model = RandomForestRegressor(max_depth=rf_max_depth, 
                                                       n_estimators=10)
    elif regressor_name == "DecisionTreeRegressor":
        dt_criteria = trial.suggest_categorical('criterion', ['mse', 'friedman_mse'])
        dt_max_depth = trial.suggest_int("dt_max_depth", 2, 12)
        model = DecisionTreeRegressor(criterion= dt_criteria,
                                                   max_depth=dt_max_depth)
    elif regressor_name == 'LinearRegression':
        model = LinearRegression() 
    

    scaler_string = trial.suggest_categorical("------------------------------------_scaler",["no_scaler", "StandardScaler","RobustScaler","MinMaxScaler", "MaxAbsScaler", "StandardScaler", "PowerTransformer","Normalizer"])    
    if scaler_string == "no_scaler":
        scaled_X = X
    else:
        scaler = eval(scaler_string)()
        scaler.fit(X)
        scaled_X = scaler.transform(X)

    cv=KFold(shuffle=True,random_state=42)
    score=cross_val_score(model,scaled_X,y,cv=cv,scoring="neg_mean_absolute_error")
    trial_score = score.mean() 

    return trial_score

X,y = load_data()
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)
print(study.best_trial)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pickle

class PredictionCharges():
    MODEL_SAVE_LOCATION = "/work/data/homework 25/"
    # PATH = "data/insurance.csv"

    def __init__(self):
        self.df = self.load_data()
        scaled_X, y = self.preprocessing()
        self.train_best_model(scaled_X, y)
        self.save_model()
        
    def load_data(self):
        return pd.read_csv(self.MODEL_SAVE_LOCATION + "insurance.csv")

    def preprocessing(self):
        X=self.df.drop(["charges"],axis = 1)
        y=self.df["charges"]
        self.ct = ColumnTransformer(transformers=[
                                            ('one_hot_encoder', OneHotEncoder(), [1, 4, 5])], 
                                            remainder='passthrough')
        # self.ct.fit(X)
        # X = np.array(self.ct.transform(X))                                    
        X = np.array(self.ct.fit_transform(X))
        
        self.scaler = StandardScaler()
        # self.scaler.fit(X)
        # scaled_X = self.scaler.transform(X)
        scaled_X = self.scaler.fit_transform(X)
        return scaled_X, y

    def train_best_model(self, scaled_X, y):
        self.model = RandomForestRegressor(max_depth=6)
        self.model.fit(scaled_X, y)
    
    def save_model(self):
        pickle.dump(self.model,  open(self.MODEL_SAVE_LOCATION +"classification_model.joblib", 'wb'))
        pickle.dump(self.ct,  open(self.MODEL_SAVE_LOCATION + "col_transformer.joblib", 'wb'))
        pickle.dump(self.scaler,  open(self.MODEL_SAVE_LOCATION + "scaler.joblib", 'wb'))

    def predict(self, value_to_predict):
        df= pd.DataFrame([value_to_predict],columns=["age", "sex", "bmi", "children", "smoker", "region"])

        loaded_ct = pickle.load(open(self.MODEL_SAVE_LOCATION + "col_transformer.joblib", 'rb'))
        loaded_scaler = pickle.load(open(self.MODEL_SAVE_LOCATION + "scaler.joblib", 'rb'))
        loaded_model = pickle.load(open(self.MODEL_SAVE_LOCATION + "classification_model.joblib", 'rb'))

        X = np.array(loaded_ct.transform(df))
        X = loaded_scaler.transform(X)

        prediction = loaded_model.predict(X)[0]
        return round(prediction,2)

model = PredictionCharges()
model.predict([20, "male", 30, 1,  "no", "southwest"])


3334.57

In [ ]:
def load_data():
    X=df.drop(["charges"],axis = 1)
    y=df["charges"]
    le = LabelEncoder()
    X.iloc[:, 1] = le.fit_transform(X.iloc[:, 1])
    X.iloc[:, 4] = le.fit_transform(X.iloc[:, 4])
    ct = ColumnTransformer(transformers=[('OneHotEncoder', OneHotEncoder(), [5])], 
                            transformers=[('label', LabelEncoder(), [2])], 
                        remainder='passthrough')
    X = np.array(ct.fit_transform(X))   
    return X,y

X,y = load_data()    


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7d3ce7c8-a514-49e4-9ba4-a5899ac52ea5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>